In [247]:
import pandas as pd
from pandas import DataFrame, read_csv, Series
import numpy as np
import math
import sklearn.datasets
from sklearn.cluster import KMeans
import nltk
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

In [248]:
# получение данных о студентах и их оценках по дисциплинам
import csv
def read_file(filename):
    f = open(filename)
    data = csv.reader(f)
    mentions = dict()
    for line in data:
        student = line[0]
        discipline = line[6]
        type_d = line[7]
        sem_rate = int(line[8])
        bonus = int(line[9])
        exam_rate = int(line[10])
        rate = 0
        if type_d == 'exam':
            if (sem_rate >= 38) & (exam_rate >= 22):
                rate = sem_rate + exam_rate + bonus
        else: 
            if (sem_rate >= 60):
                rate = sem_rate + bonus
        if not student in mentions:
            mentions[student] = dict()
        mentions[student][discipline] = rate
    f.close()
    return mentions

In [249]:
# косинусная мера сходства: насколько два пользователя похожи друг на друга. 
def dist_cosine(vecA, vecB):
    def dot_product(vecA, vecB):
        d = 0.0
        for dim in vecA:
            if dim in vecB: 
                d += vecA[dim]*vecB[dim]
        return d
    a = dot_product(vecA,vecB)
    b = math.sqrt(dot_product(vecA,vecA))
    c = math.sqrt(dot_product(vecB,vecB))
    if (a == 0) or (b == 0) or (c == 0):
        return 0
    else:
        return  (a / (b * c))

In [250]:
def similar_students(studentID, studentsData, nBestStudents = 3):
    matches = [(u, dist_cosine(studentsData[studentID], studentsData[u])) for u in studentsData.keys() if u != studentID]
    bestMatches = sorted(matches, key=lambda q:(q[1], q[0]), reverse=True)[:nBestStudents]
    print("Most correlated with studentID: " + str(studentID))
    for line in bestMatches:
        print("StudentID: " + str(line[0]) + "  Coeff: " + str(line[1])) 
    return [(x[0], x[1]) for x in bestMatches]

In [251]:
students_test = read_file("raw_data32.csv")
#df = DataFrame(students_test)
#df.columns.name = 'studentID'
#df.index.name = 'discipline'    
rec = similar_students('1', students_test, 10)


Most correlated with studentID: 1
StudentID: 9  Coeff: 0.7542152087381664
StudentID: 6  Coeff: 0.7248624347806895
StudentID: 7  Coeff: 0.7194365076928019
StudentID: 5  Coeff: 0.6837763094049188
StudentID: 4  Coeff: 0.5597650673256156
StudentID: 2  Coeff: 0.22271363971105235
StudentID: 3  Coeff: 0.06705017139158977
StudentID: 8  Coeff: 0


In [252]:
# Похожесть дисциплин
def read_subjects(filename):
    f = open(filename)
    data = csv.reader(f)
    subjects = dict()
    for line in data:
        ID = line[0]
        discipline = line[1]
        subjects[ID] = discipline
    f.close()
    return subjects

In [253]:
#sd = read_subjects("disciplines2.csv")

#nltk.download() 

stopwords = set(stopwords.words("russian"))
#print(stopwords)


In [254]:
def review_to_words(review_text):
    review_text = review_text.replace('ё','е')
    review_text = review_text.replace('Ё','Е')
    # 1. Удаляем все, кроме букв
    letters_only = re.sub("[^а-яА-Я]", " ", review_text) 
    # 2. Делаем все буквы строчными и создаем массив слов
    words = letters_only.lower().split()                             
    # 3. Удаляем незначащие слова
    meaningful_words = [w for w in words if not w in stopwords]
    # 4. Формируем текст, объединяя слова через пробел
    return(" ".join(meaningful_words))

In [255]:
d0 = pd.read_csv("disciplines.csv", encoding='utf-8')
d0.head(3)

,ID,Name
0,1,Алгоритмы на графах
1,2,Функциональное программирование
2,3,Функциональное программирование


In [256]:
d1 = d0.copy()
d1['Name'] = d1['Name'].apply(lambda s: review_to_words(s))
d = d1.sort_values('Name')
d = d.Name.unique()
d
# кластеризовать уникальные имена, потом смотреть, какому кластеру принадлежит id дисциплины, а затем посмотреть схожие дисциплины

array(['автоматизация торгового предприятия базе', 'адаптация',
       'адвокатура рф', 'административное право',
       'администрирование вычислительных кластеров',
       'администрирование серверных систем базе виндовс',
       'актуальные проблемы вещного права',
       'актуальные проблемы гидромеханики',
       'актуальные проблемы жилищного права',
       'актуальные проблемы интеллектуальной собственности',
       'актуальные проблемы обязательственного права',
       'актуальные проблемы ответственности гражданском праве',
       'актуальные проблемы теории права',
       'актуальные проблемы уголовно исполнительного права',
       'актуальные проблемы уголовного права',
       'актуальные проблемы уголовного права ч',
       'актуальные проблемы уголовного процесса',
       'актуальные теоретические методологические проблемы гражданского права',
       'актуарная математика', 'алгебра', 'алгебра геометрия',
       'алгебра линейных операторов конечномерных линейных пространс

In [257]:
#связываем плохие и хорошие имена в словаре dict_dis
d3 = d0.copy().sort_values('Name').Name.unique()
dict_dis = dict()
it = 0
for s in d3:
    word = review_to_words(s)
    if not word in dict_dis:
        dict_dis[word] = s
        it += 1
dict_dis

{'автоматизация торгового предприятия базе': 'Автоматизация торгового предприятия на базе 1С',
 'адаптация': 'Адаптация 1С',
 'адвокатура рф': 'Адвокатура в РФ',
 'административное право': 'Административное право',
 'администрирование вычислительных кластеров': 'Администрирование вычислительных кластеров',
 'администрирование серверных систем базе виндовс': 'Администрирование серверных систем на базе Виндовс',
 'актуальные проблемы вещного права': 'Актуальные проблемы вещного права',
 'актуальные проблемы гидромеханики': 'Актуальные проблемы гидромеханики',
 'актуальные проблемы жилищного права': 'Актуальные проблемы жилищного права',
 'актуальные проблемы интеллектуальной собственности': 'Актуальные проблемы интеллектуальной собственности',
 'актуальные проблемы обязательственного права': 'Актуальные проблемы обязательственного права',
 'актуальные проблемы ответственности гражданском праве': 'Актуальные проблемы ответственности в гражданском праве',
 'актуальные проблемы теории права

In [258]:
#vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 50000, binary = False) 
#vectorized = vectorizer.fit_transform(d1['Name'])
##print(vectorized)

#num_samples, num_features = vectorized.shape
## print((num_samples, num_features)) # 918 дисциплин, 1123-мерный вектор признаков

#vocabulary = vectorizer.get_feature_names()
##print(vocabulary[20])

#num_clusters = 50
#km = KMeans(n_clusters=num_clusters, init='random', n_init=10, verbose=1, random_state=3)
#km.fit(vectorized)


In [259]:
#km.predict(vectorized)
#km.cluster_centers_

In [260]:
##print(km.cluster_centers_)
#print(km.labels_)

In [261]:
#d1['Name'][km.labels_ == 46]

In [262]:
# стемминг
from nltk.stem.snowball import SnowballStemmer
# print(SnowballStemmer("russian").stem("гениально"))  # проверка работы стемминга

In [263]:
rus_stemmer = SnowballStemmer('russian')

class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: (rus_stemmer.stem(w) for w in analyzer(doc))
    
vectorizer = StemmedCountVectorizer(min_df=1, analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = None, binary = False)
#vectorized = vectorizer.fit_transform(d1['Name'])
vectorized = vectorizer.fit_transform(d)

In [264]:
num_samples, num_features = vectorized.shape
vocabulary = vectorizer.get_feature_names()

num_clusters = 450
km = KMeans(n_clusters=num_clusters, init='random', n_init=13, verbose=1, random_state=3)
km.fit(vectorized)

Initialization complete
Iteration  0, inertia 515.000
Iteration  1, inertia 299.646
Iteration  2, inertia 292.102
Converged at iteration 2: center shift 0.000000e+00 within tolerance 5.323907e-07
Initialization complete
Iteration  0, inertia 536.000
Iteration  1, inertia 307.151
Iteration  2, inertia 292.799
Iteration  3, inertia 291.899
Converged at iteration 3: center shift 0.000000e+00 within tolerance 5.323907e-07
Initialization complete
Iteration  0, inertia 542.000
Iteration  1, inertia 302.104
Iteration  2, inertia 286.304
Iteration  3, inertia 285.512
Converged at iteration 3: center shift 0.000000e+00 within tolerance 5.323907e-07
Initialization complete
Iteration  0, inertia 537.000
Iteration  1, inertia 296.935
Iteration  2, inertia 286.061
Converged at iteration 2: center shift 0.000000e+00 within tolerance 5.323907e-07
Initialization complete
Iteration  0, inertia 515.000
Iteration  1, inertia 291.187
Iteration  2, inertia 279.844
Converged at iteration 2: center shift 0.0

KMeans(algorithm='auto', copy_x=True, init='random', max_iter=300,
    n_clusters=450, n_init=13, n_jobs=1, precompute_distances='auto',
    random_state=3, tol=0.0001, verbose=1)

In [265]:
km.predict(vectorized)
km.cluster_centers_

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [266]:
for label in km.labels_: 
    print(label, end = " ")

242 224 427 155 154 115 118 381 413 381 118 91 415 156 193 193 157 91 134 59 279 352 85 239 77 47 47 7 333 45 417 395 179 197 197 207 72 149 131 167 109 411 205 411 233 37 391 253 58 449 447 147 331 327 15 115 170 363 59 170 170 223 223 166 9 97 316 372 339 409 164 444 323 42 172 417 222 160 406 213 429 232 422 221 151 248 441 356 171 311 421 167 228 208 173 159 390 280 13 10 10 28 358 105 358 34 34 43 371 356 159 169 312 276 291 364 140 8 113 4 218 254 88 140 75 2 32 332 79 289 173 185 185 32 231 299 80 335 123 107 428 368 292 272 410 120 192 85 385 345 403 39 161 373 345 117 27 373 150 100 19 407 2 274 286 137 377 2 235 271 270 267 137 145 267 328 137 66 435 259 130 418 168 308 257 257 303 89 78 56 73 56 56 257 393 133 225 51 32 442 423 403 361 341 354 348 353 32 27 27 2 351 265 2 5 36 230 326 398 2 317 106 253 56 341 326 1 215 386 347 386 347 177 420 2 11 1 68 2 352 2 325 158 92 406 105 105 367 349 151 445 263 244 136 357 64 132 357 132 64 81 93 123 241 300 60 275 256 284 99 256 211

In [267]:
#d1['Name'][km.labels_ == 0]
d[km.labels_ == 389]
    # к прошлым данным:
    # 25 кластер "очень плохой" при init = random, num_kluster = 500, n_init = 13, неправильно соотносит предмет "Теория графов"
    # 41 кластер "плохой" при init = k_means++, num_kluster = 500, n_init = 13
    # 62 кластер "плохой" при init = k_means++, num_kluster = 523, n_init = 13
    # хорошое разбиение при init = k_means++, num_kluster = 700, n_init = 13, но очень много "одиночных" дисциплин
    # 23 кластер "плохой" при init = k_means++, num_kluster = 600, n_init = 13
    # 79 кластер "плохой" при init = k_means++, num_kluster = 600, n_init = 15
#d1['Name'][315]
#d1['Name'][km.labels_ == 56].keys() #id дисципли

array(['методы анализа организаций моделирования бизнес процессов'], dtype=object)

In [268]:
import scipy as sp

def disciplines_in_one_cluster(from_info, discipline):
    discipline = review_to_words(discipline)
    discipline_vec = vectorizer.transform([discipline])
    discipline_label = km.predict(discipline_vec)
    return discipline_label[0] #вернуть номер кластера, в котором располагаются схожие дисциплины
    #return from_info[km.labels_ == discipline_label] #вернуть схожие дисциплины 

In [269]:
print("Test 1")
#sim_dis_info = disciplines_in_one_cluster(d1['Name'], "Теория графов")
sim_dis_info = disciplines_in_one_cluster(d, "Теория графов")
print(sim_dis_info)
print()

print("Test 2")
#sim_dis_info = disciplines_in_one_cluster(d1['Name'], "Компьютерные сети")
sim_dis_info = disciplines_in_one_cluster(d, "Компьютерные сети")
print(sim_dis_info)
print()

print("Test 3")
#sim_dis_info = disciplines_in_one_cluster(d1['Name'], "Механика")
sim_dis_info = disciplines_in_one_cluster(d, "Механика")
print(sim_dis_info)
print()

print("Test 4")
#sim_dis_info = disciplines_in_one_cluster(d1['Name'], "Алгоритмы на графах")
sim_dis_info = disciplines_in_one_cluster(d, "Алгоритмы на графах")
print(sim_dis_info)

Test 1
18

Test 2
393

Test 3
119

Test 4
47


In [270]:
# stud_rating - словарь : дисциплина - оценка
# d_name - все "испорченные" наименования дисциплин c id  -- dataframe
# good_dis - все "НЕиспорченные" наименования дисциплин c id  -- dict
# rating - словарь : кластер - суммарная оценка
# num_rating - словарь : кластер - кол-во предметов из данного кластера, пройденных студентом 

# ф-ия находит среднее значение баллов, полученных студентом, для каждого кластера
# результирующий rating - словарь : кластер - средний балл

def average_rating_in_clusters(stud_rating, d_name, good_dis):
    rating = dict()
    num_rating = dict()
    for elem in stud_rating:
        num_clust = disciplines_in_one_cluster(d_name, good_dis[elem])
        if not num_clust in rating:
            rating[num_clust] = stud_rating[elem]
            num_rating[num_clust] = 1
        else:
            rating[num_clust] += stud_rating[elem]
            num_rating[num_clust] += 1
    for i in rating:
        rating[i] = rating[i] / num_rating[i];
    return rating

In [271]:
# Наименование всех пройденных студентом дисциплин
def passed_subjects(stud_rating, good_dis):
    l = []
    for i in stud_rating:
        l.append(good_dis[i])
    return l

In [272]:
# rating - словарь : кластер - средний балл
# p_sub - результат ф-ии passed_subjects
# from_bad_to_good_name - см.выше dict_dis
# info_cluster
# res_disciplines - дисциплины из "хороших кластеров" - средний балл больше 85, которые студент еще не прошел
def add_disciplines(rating, p_sub, from_bad_to_good_name, info_cluster):
    res_disciplines = []
    for i in rating: 
        if (rating[i] > 85):
            for j in info_cluster[km.labels_ == i]:
                good = from_bad_to_good_name[j]
                if good not in p_sub:
                    res_disciplines.append(good)
    return res_disciplines

In [273]:
# рекомендации для студента id_student
# информацию о студентах получаем из students_info
# выбираем предметы, основываясь на схожести с n_best_students наиболее похожими студентами
# good_dis - все "НЕиспорченные" наименования дисциплин c id  -- dict
# d_name - все "испорченные" наименования дисциплин c id  -- dataframe
def make_recommendations(id_student, students_info, d_name, good_dis, from_bad_to_good_name, n_best_students = 3):
    sim_stud = similar_students(id_student, students_info, n_best_students)  # нахождение похожих студентов
    list_passed = passed_subjects(students_info[id_student], good_dis)       # список наименований предметов,изученных студентом
    av_rating = average_rating_in_clusters(students_info[id_student], d_name, good_dis) # список среднего балла для каждого кластера, в котором хотя бы один предмет был изучен студентом
    rec_disciplines = add_disciplines(av_rating, list_passed, from_bad_to_good_name, d_name) # если средний балл в кластере больше 85, то добавляем из кластера предметы, которые студент еще не прошел
    # ============= блок работы с похожими студентами ============= 
    for i in sim_stud:
        av = average_rating_in_clusters(students_info[i[0]], d_name, good_dis) # список среднего балла для каждого кластера, в котором хотя бы один предмет был изучен студентом i[0]
        stud_disciplines = rec_disciplines + list_passed # список предметов, которые уже посоветовали студенту id_student + студент id_student прошел сам 
        similar_stud_rec = add_disciplines(av, stud_disciplines, from_bad_to_good_name, d_name) # если средний балл в кластере больше 85, то добавляем из кластера предметы, которые студент id_student еще не прошел
        rec_disciplines += similar_stud_rec # все рекомендации для студента id_student
    # ============= блок работы с похожими студентами закончен ============= 
    return rec_disciplines
    


In [274]:
sd = read_subjects("disciplines2.csv")
students_rating = read_file("students_rating.csv")
rec = make_recommendations('11', students_rating, d, sd, dict_dis, 3)
rec

Most correlated with studentID: 11
StudentID: 5  Coeff: 0.9729210307385605
StudentID: 18  Coeff: 0.9683713392884907
StudentID: 15  Coeff: 0.967386392071021


['Практикум на ЭВМ (компьютерные сети)',
 'История физики',
 'Группы Ли и дифференциальные уравнения',
 'Компьютерная графика',
 'Компьютерные науки',
 'Философия и методология науки',
 'Стохастический анализ',
 'Исследование операций и методы оптимизации',
 'Методология и методы научного исследования',
 'Основы математической обработки информации',
 'Алгебраические и дискретные структуры информатики',
 'Информатика и ИКТ',
 'Методология дидактических исследований в информатике',
 'Основы квалификации преступлений',
 'Основы механики стенок кровеносных сосудов',
 'Дискретная математика в приложениях и задачах',
 'Избранные разделы математического анализа',
 'Алгебра линейных операторов в конечномерных линейных пространствах']